In [1]:
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, OWL
from owlrl import DeductiveClosure, OWLRL_Semantics
from intension import Intension
from tqdm import tqdm
import pandas as pd
import json, os, random

In [16]:
MODELS = [ 
    # { "model_name": "gpt-3.5-turbo", "batch_size": 50 },
    { "model_name": "gpt-4o-2024-05-13", "batch_size": 50 },
    { "model_name": "gpt-4-0125-preview", "batch_size": 50 },
    { "model_name": "mistralai/Mistral-7B-Instruct-v0.3", "batch_size": 50 },
    { "model_name": "claude-3-5-sonnet-20240620", "batch_size": 1 },
    { "model_name": "mistralai/Mixtral-8x7B-Instruct-v0.1", "batch_size": 50 },
    { "model_name": "claude-3-opus-20240229", "batch_size": 1 },
    # { "model_name": "meta-llama/Meta-Llama-3-70B-Instruct", "batch_size": 50 },
    { "model_name": "claude-3-haiku-20240307", "batch_size": 1 },
]

In [3]:
TEST_TRIPLES_SAMPLE_SIZE = 10

In [8]:
# Define vrd: namespace
VRD = Namespace("http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#")

# Get Turtle serialization of vrd_world_v1.owl
VRD_WORLD_OWL = open("NeSy4VRD/nesy4vrd_ontology/vrd_world_v1.owl", "r").read()

# Create a new graph
graph = Graph()
graph.bind("vrd", VRD)
graph.bind("rdf", RDF)
graph.bind("rdfs", RDFS)
graph.bind("owl", OWL)
graph.parse("NeSy4VRD/nesy4vrd_ontology/vrd_world_v1.owl", format='turtle')

# Create another graph to store deductive closure
closure = Graph()
graph.bind("vrd", VRD)
closure.bind("rdf", RDF)
closure.bind("rdfs", RDFS)
closure.bind("owl", OWL)
closure += graph
DeductiveClosure(OWLRL_Semantics).expand(closure)

# Create a graph to store set difference between closure and graph (i.e. the inferred triples)
inferred = Graph()
graph.bind("vrd", VRD)
inferred.bind("rdf", RDF)
inferred.bind("rdfs", RDFS)
inferred.bind("owl", OWL)
inferred += (closure - graph)

# Sample from the set of inferred triples
inferred_triples = list(inferred)
if TEST_TRIPLES_SAMPLE_SIZE < len(inferred_triples):
    inferred_triples = random.sample(inferred_triples, TEST_TRIPLES_SAMPLE_SIZE)

# Create a graph to store the sample
test = Graph()
test.bind("vrd", VRD)
test.bind("rdf", RDF)
test.bind("rdfs", RDFS)
test.bind("owl", OWL)
for triple in inferred_triples:
    test.add(triple)

# Print test triples
for s, p, o in test.triples((None, None, None)):
    print(f'<{s}>, <{p}>, <{o}>')


<http://www.w3.org/2001/XMLSchema#positiveInteger>, <http://www.w3.org/2002/07/owl#sameAs>, <http://www.w3.org/2001/XMLSchema#positiveInteger>
<http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#Bus>, <http://www.w3.org/2000/01/rdf-schema#subClassOf>, <http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#VRDWorldThing>
<http://www.w3.org/2002/07/owl#Nothing>, <http://www.w3.org/2000/01/rdf-schema#subClassOf>, <http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#FlyableThing>
<http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#carry>, <http://www.w3.org/2000/01/rdf-schema#domain>, <nf28ef2ac4d834d6e80f82fa5f3eec9c2b13>
<http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#walk>, <http://www.w3.org/2000/01/rdf-schema#domain>, <http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#MotionCapableThing>
<http://www.semanticweb.org/nesy4vrd/ontologies/vrd_world#standUnder>, <http://www.w3.org/2000/01/rdf-schema#domain>, <http://www.w3.org/2002/07/owl#Thing>
<http:/

In [11]:
queries = [ { "s": str(s), "p": str(p), "o": str(o), "graph": VRD_WORLD_OWL } for s, p, o in test.triples((None, None, None)) ]

In [17]:
for model in MODELS:
    filename = f'experiments/nesy4vrd/{model["model_name"].split("/")[-1]}-owl-inf.json'
    if os.path.isfile(filename):
        print(f'{model["model_name"]:36}: EXISTS')
    else:
        results = []
        batches = [ queries[i:i+model["batch_size"]] for i in range(0, len(queries), model["batch_size"]) ] 
        intension = Intension(model=model["model_name"])
        for batch in tqdm(batches, desc=f'{model["model_name"]:36}', total=len(batches)):
            response = intension.chain.batch(batch)
            for i, result in enumerate(response):
                result["model"] = model["model_name"]
                result["rationale"] = result["text"]["rationale"]
                result["answer"] = result["text"]["answer"]
                result.pop("text")
            results.extend(response)
        json.dump(results, open(filename, "w+"))

gpt-4o-2024-05-13                   : EXISTS
gpt-4-0125-preview                  : EXISTS
mistralai/Mistral-7B-Instruct-v0.3  : EXISTS
claude-3-5-sonnet-20240620          : EXISTS
mistralai/Mixtral-8x7B-Instruct-v0.1: EXISTS
claude-3-opus-20240229              : EXISTS


claude-3-haiku-20240307             : 100%|██████████| 10/10 [00:30<00:00,  3.08s/it]
